In [64]:
from math import ceil
from sklearn.utils import shuffle
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import model_selection 
import seaborn as sns 
from sklearn.model_selection import StratifiedKFold 
from sklearn import metrics
from numpy import mean
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import 
import torch.nn.functional as nnf
from sklearn.tree import DecisionTreeClassifier
from fastai.tabular.all import *
import os

In [65]:

def getDataframe(p):
    print(f"file name : {os.path.split(p)[1]}")
    if os.path.split(p)[1] == 'JM1.csv':
        df = pd.read_csv(p)
        df['label'] = df['label'].map({"b'N'" :False,"b'Y'" :True})
        return df
    else:
        df = pd.read_csv(p)
        df['Defective'] = df['Defective'].map({"b'N'" :False,"b'Y'" :True})
        return df


# preprecessing 

def normalize(df):
    result = df.copy()
    for col in df.iloc[:,:-1].columns:
        max_value = df[col].max()
        min_value = df[col].min()
        result[col] = (df[col] - min_value) / (max_value - min_value)
    # print(df.head())
    return result

def preprocessing(df):
    # select k best columns
    best_features = SelectKBest(score_func=chi2,k=10)
    fit = best_features.fit(df.iloc[:,:-1],df.iloc[:,-1])

    dfscores = pd.DataFrame(fit.scores_)
    df_cols = pd.DataFrame(df.iloc[:,:-1].columns)

    feature_scores = pd.concat([df_cols,dfscores],axis=1)
    feature_scores.columns = ['parameters','score']
    selected_features = feature_scores.nlargest(10,'score').parameters.values
    df = df.drop(selected_features, axis=1)
    # normalize the data
    df = normalize(df)
    return df




# making bags
def sampling(df):
    # create 2 sets of majority and minority samples
    a = df[df.iloc[:,-1] == False]
    b = df[df.iloc[:,-1] == True]
    majority = None
    minority = None
    if a.shape[0] > b.shape[0]:
        # print("here")
        majority = a
        minority = b
    else:
        # print("there")
        minority = b
        majority = a
    # print(majority.head())
    # print(minority.head())

    # no of bags 
    bags = ceil(majority.shape[0]/minority.shape[0]) + 2
    print(f"majority cnt :{majority.shape[0]} , minority cnt :{minority.shape[0]} and number of bags: {bags}")
    subsets = []
    for i in range(bags):
        maj_sample = majority.sample(n=minority.shape[0])
        balanced_subset = pd.concat([minority,maj_sample])
        # print(maj_sample.shape)
        # print(balanced_subset.shape)
        balanced_subset = shuffle(balanced_subset)
        # print(balanced_subset.head())
        print(balanced_subset.shape)
        subsets.append(balanced_subset)

    return subsets

    
# train  
def trainModels(balanced_subsets):
    trained_models = []
    for subset in balanced_subsets:
        df = subset
        # cont_names = df.columns
        cat_names = []
        procs=[]
        df1 = subset
        cont_names = ['LOC_BLANK', 'LOC_CODE_AND_COMMENT', 'LOC_COMMENTS',
            'CYCLOMATIC_COMPLEXITY', 'DESIGN_COMPLEXITY',
            'ESSENTIAL_COMPLEXITY', 'HALSTEAD_CONTENT', 'HALSTEAD_DIFFICULTY',
            'HALSTEAD_ERROR_EST', 'HALSTEAD_LEVEL', 'NUM_UNIQUE_OPERATORS',
            'label']
        cat_names = []
        procs=[]
        # config = tabular_config(embed_p=0.6, use_bn=False); config
        dls = TabularDataLoaders.from_df(df1,path='.',procs=None,cont_names=cont_names[:-1],cat_names=None,y_names=cont_names[-1])
        learn = tabular_learner(dls,[400,200,100,50],metrics=[accuracy,RocAucBinary()])
        learn.fit_one_cycle(10,1e-3)
        # learn.show_training_loop()
        trained_models.append(learn)
    return trained_models

def majorityVoteWieghtage(trained_models, testSet):
    # finalpreds = []
    fp = []
    probs = []
    for model in trained_models:
        dl_test = model.dls.test_dl(testSet, with_labels=True)
        preds = model.get_preds(dl=dl_test)
        # print(preds)
        fp.append(preds[1].T.squeeze())
        probs.append(preds[0].T.squeeze())
    # print(probs[0].shape)
    t1 = torch.stack(fp)
    t1 = t1.T
    p1 = torch.stack(probs)
    p1 = p1.T
    # print(t1.shape)
    # print(t1)
    # print(p1.shape)
    # print(p1)
    p1 = torch.mean(p1,dim=2)
    p1 = p1.squeeze()
    prob = nnf.softmax(p1, dim=1)
    top_p, top_class = prob.topk(1, dim = 1)

    return top_class.squeeze(),p1
        
        

In [66]:
df = getDataframe("Dataset/JM1.csv")


# splitting data into train and test sets
train,testSet = train_test_split(df,test_size=0.2)
# print(df.head())
print(train.head(),testSet.head())

# create balanced subset datasets having all minority sets and a random sample from majority set.
balanced_subsets = sampling(train)


# get all trained models from all the above balanced subsets
trained_models = trainModels(balanced_subsets)

minorityClassifierPreds,minorityClassifierProbs = majorityVoteWieghtage(trained_models,testSet)





file name : JM1.csv
      LOC_BLANK  BRANCH_COUNT  LOC_CODE_AND_COMMENT  LOC_COMMENTS  \
8596        0.0           1.0                   0.0           0.0   
5580        7.0          29.0                   0.0          10.0   
9586        2.0           7.0                   0.0           0.0   
3072        0.0           1.0                   0.0           0.0   
7722        0.0           1.0                   0.0           0.0   

      CYCLOMATIC_COMPLEXITY  DESIGN_COMPLEXITY  ESSENTIAL_COMPLEXITY  \
8596                    1.0                1.0                   1.0   
5580                   15.0               14.0                   1.0   
9586                    4.0                4.0                   1.0   
3072                    1.0                1.0                   1.0   
7722                    1.0                1.0                   1.0   

      LOC_EXECUTABLE  HALSTEAD_CONTENT  HALSTEAD_DIFFICULTY  ...  \
8596             2.0              7.74                 1.50  ...

In [21]:
minorityClassifierProbs

tensor([[0.3140, 0.6860],
        [0.2910, 0.7090],
        [0.6407, 0.3593],
        ...,
        [0.2551, 0.7449],
        [0.6795, 0.3205],
        [0.4156, 0.5844]])

In [22]:
def majTrain(train):
    cont_names = ['LOC_BLANK', 'LOC_CODE_AND_COMMENT', 'LOC_COMMENTS',
            'CYCLOMATIC_COMPLEXITY', 'DESIGN_COMPLEXITY',
            'ESSENTIAL_COMPLEXITY', 'HALSTEAD_CONTENT', 'HALSTEAD_DIFFICULTY',
            'HALSTEAD_ERROR_EST', 'HALSTEAD_LEVEL', 'NUM_UNIQUE_OPERATORS',
            'label']
    cat_names = []
    procs=[]
    dls = TabularDataLoaders.from_df(train,path='.',procs=None,cont_names=cont_names[:-1],cat_names=None,y_names=cont_names[-1])
    learn = tabular_learner(dls,[400,200,100,50],metrics=[accuracy,RocAucBinary()])
    learn.fit_one_cycle(10,1e-3)
    return learn

def majPrediction(model,testSet):
    dl_test = model.dls.test_dl(testSet, with_labels=true)
    preds = model.get_preds(dl=dl_test)
    return preds
    

In [23]:
# train majority for majority class

# maj_trained_model = trainModels([train])
majorityClassifier = majTrain(train)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,0.474104,0.449904,0.823990,0.664886,00:01
1,0.445348,0.436814,0.823338,0.697301,00:01
2,0.435559,0.435815,0.823990,0.685224,00:01
3,0.429028,0.434873,0.820730,0.690106,00:00
4,0.422811,0.437623,0.825293,0.686757,00:01
5,0.424173,0.436981,0.825293,0.686253,00:01
6,0.423023,0.440109,0.823990,0.686701,00:01
7,0.419479,0.442232,0.825293,0.676350,00:01
8,0.414003,0.440443,0.821382,0.683720,00:01
9,0.413964,0.441496,0.822686,0.681152,00:01


In [24]:
preds = majPrediction(majorityClassifier,testSet)

In [25]:
preds

(tensor([[0.6933, 0.3067],
         [0.7381, 0.2619],
         [0.8456, 0.1544],
         ...,
         [0.7026, 0.2974],
         [0.9061, 0.0939],
         [0.7796, 0.2204]]),
 tensor([[0],
         [0],
         [1],
         ...,
         [1],
         [0],
         [0]], dtype=torch.int8))

In [35]:
testSet.iloc[:,-1]

8704    False
1887    False
604      True
9364    False
358      True
        ...  
252      True
9148    False
755      True
7944    False
2618    False
Name: label, Length: 1919, dtype: bool

In [45]:
(torch.tensor([0.69,0.30]) + torch.tensor([0.5,0.5]))/2

tensor([0.5950, 0.4000])

In [50]:
def meta(mnPred,mnProbs,mjPreds,mjprobs,Wmn,Wmj):
    final_preds = torch.zeros_like(mnPred)
    for i in range(mnPred.shape[0]):
        if mjPreds[i] == 1:
            final_preds[i] = 1
        elif mnPred[i] == 0:
            final_preds[i] = 0
        else:
            z = (mnProbs[i]*Wmn + mjprobs[i]*Wmj)/2
            # print(z)
            if(z[0] > z[1]):
                final_preds[i] = 0
            else:
                final_preds[i] = 1
    return final_preds

In [58]:
predictions = meta(minorityClassifierPreds,minorityClassifierProbs,preds[1],preds[0],1.5,1)

In [59]:
predictions.shape

torch.Size([1919])

In [55]:
from sklearn.metrics import roc_curve

# roc curve for modelsa
fpr1, tpr1, thresh1 = roc_curve(testSet.iloc[:,-1], preds[1], pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(testSet.iloc[:,-1], minorityClassifierPreds, pos_label=1)
fpr3, tpr3, thresh3 = roc_curve(testSet.iloc[:,-1], predictions, pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(testSet.iloc[:,-1]))]
p_fpr, p_tpr, _ = roc_curve(testSet.iloc[:,-1], random_probs, pos_label=1)

In [61]:
# matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# plot roc curves
# plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Majority DNN')
# plt.plot(fpr2, tpr2, linestyle='--',color='green', label='Minority DNN')
plt.plot(fpr3, tpr3, linestyle='--',color='red', label=' fp')

plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

In [63]:
from sklearn.metrics import roc_auc_score

# auc scores
auc_score1 = roc_auc_score(testSet.iloc[:,-1], predictions)
# auc_score2 = roc_auc_score(testSet.iloc[:,-1], )

print(auc_score1)

0.9508928571428572


In [110]:
net

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=12, out_features=400, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=400, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (3): LinBnDrop(
      (0): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=100, out_features=50,